In [18]:
# This notebook is used to write the scripts and then write them into shell scripts - isn't required for the pipeline

# GIF: Nicolas Cage Montage
---
Combine a sequence of images of Nicolas Cage's face, sorted by smartgrid

In [9]:
# Goal ====================
# Directories:
    # temp - temp files
    # inputs - source images - 
    # outputs - processed images in sequence
    # movies - final movie gif
#  Scripts:
    # build-directories.sh - script to create folders
    # run-cage.sh - used to run the pipeline as outlined below
        # get-cage.sh - scrapes images of nicolas cage and puts them into inputs
        # process-cage.sh - goes through input images and filters out non-cage photos + resizes to 440 X 220
        # sort-cage.sh - use smartgrid to sort images and put them into outputs
        # composite-cage - use smartgrid to sort images and put them into outputs    

In [28]:
# Build Directories ===================
build_dirs = 'mkdir temp inputs outputs movies'
! {build_dirs}

mkdir: temp: File exists
mkdir: inputs: File exists
mkdir: outputs: File exists
mkdir: movies: File exists


In [29]:
# Misc. helpers
def write_script(title, content, extension) :
    f = open('{}.{}'.format(title, extension), 'w')
    f.write(content)
    f.close()

In [30]:
# Get Nick Cage photos ================
# source: http://niccageaseveryone.blogspot.co.nz
get_cage = '\
set -ex\
\nfor YEAR in 2009 2010 2011 2013\
\ndo\
\n\tfor PAGE in $(seq -f "%02g" 1 12)\
\n\tdo\
\n\t# build the url \
\n\tURL="http://niccageaseveryone.blogspot.co.nz/"$YEAR"/"$PAGE\
\n\t\t# fetch the images \\\
\n\t\twget \\\
\n\t\t\t--adjust-extension \\\
\n\t\t\t--no-directories \\\
\n\t\t\t--convert-links \\\
\n\t\t\t--backup-converted \\\
\n\t\t\t--random-wait\\\
\n\t\t\t--limit-rate=100k \\\
\n\t\t\t--span-hosts \\\
\n\t\t\t--directory-prefix=temp/downloads \\\
\n\t\t\t--page-requisites \\\
\n\t\t\t--timestamping \\\
\n\t\t\t--execute robots=off \\\
\n\t\t\t--accept=*.jpg \\\
\n\t\t\t$URL\
\n\tdone\
\ndone'

write_script("get-cage", get_cage, 'sh')

In [83]:
# Process Nick Cage photos ================
# Detect Nick Cage faces - filter out non cage faces + crop
import face_recognition
import os

# create encoding of known cage image to compare faces with
cage_image = face_recognition.load_image_file('./temp/face_recognition/to_recognise/Nicolas Cage.jpg')
cage_encoding = face_recognition.face_encodings(cage_image)[0]

# loop through downloaded images - create csv of cage face locations for cropping shell script to refer to
cage_csv = open('./temp/face_recognition/cage_bounds.csv', 'w')
column_titles = 'file_name, face_index, top, right, bottom, left'
cage_csv.write(column_titles + '\n')

for file in os.listdir('./temp/downloads'): 
    if file.endswith('.jpg'):
        image_path = os.path.join('./temp/downloads', file)
        image = face_recognition.load_image_file(image_path)
        face_locations = face_recognition.api.face_locations(image)
        if (len(face_locations) > 0): # if faces are detected
            # encode each face
            image_face_encodings = face_recognition.face_encodings(image, known_face_locations=face_locations)
            cage_locations = [] # list of bounding box coordinates for cage faces
            for index, encoding in enumerate(image_face_encodings):
                results = face_recognition.compare_faces([cage_encoding], encoding, tolerance=0.8)
                if (results[0] == True):
                    bounds = face_locations[index]
                    cage_csv.write('{}, {}, {}, {}, {}, {}\n'.format(file, index, bounds[0], bounds[1], bounds[2], bounds[3]))